In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [8]:
item = msm.convert('pdb_id:1l2y', to_form='mmtf.MMTFDecoder')

In [9]:
coordinates = np.column_stack([item.x_coord_list, item.y_coord_list, item.z_coord_list])

In [10]:
coordinates = coordinates.reshape([-1, item.num_atoms, 3])

In [11]:
coordinates.shape

(1, 11552, 3)

In [14]:
item.num_atoms

11552

In [19]:
item.num_models*304

11552

In [65]:
len(item.sec_struct_list)

760

In [7]:
coordinates = np.column_stack([item.x_coord_list, item.y_coord_list, item.z_coord_list])
coordinates = coordinates.reshape([-1, item.num_atoms, 3])
coordinates = puw.quantity(coordinates, 'angstroms')
coordinates = puw.standardize(coordinates)

11552

In [66]:
38*20

760

In [67]:
item.unit_cell

In [69]:
import numpy as np

In [70]:
aa = np.array(item.alt_loc_list)